### Installation/import dependencies

In [0]:
# Install great_expectations
!pip install 'great_expectations[spark]'
dbutils.library.restartPython()

In [0]:
# Import neccesary modules
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, DoubleType, TimestampType
from datetime import datetime
import pytz
import great_expectations as gx

In [0]:
# Set directory to reference repository
import sys
sys.path.append("/Workspace/Users/your_user@youruser.com/Git_Folder/folder_name")

# Instantiate base Class
from GX_Functions import GXRunner
gx_runner = GXRunner()

### Example Use Case

In [0]:
# Define test DataFrame
df = gx_runner.load_spark_df("catalog", "schema", "table")

In [0]:
# Register your use case Expectations
# Replace name with the name of the function you are creating
# Adapt function arguments to GX parameters required - see functions gallery on https://greatexpectations.io/expectations

gx_runner.register_expectation(
    name="expect_unique",
    func=lambda column: gx.expectations.ExpectColumnValuesToBeUnique(column=column)
)

gx_runner.register_expectation(
    name="expect_not_null",
    func=lambda column: gx.expectations.ExpectColumnValuesToNotBeNull(column=column)
)

gx_runner.register_expectation(
    name="expect_length_equal",
    func=lambda column, value: gx.expectations.ExpectColumnValueLengthsToEqual(column=column, value=value)
)

# Recover registered functions
expect_unique = gx_runner.get_expectation("expect_unique")
expect_not_null = gx_runner.get_expectation("expect_not_null")

In [0]:
# Define Expectations_Config (tests) list to validate
# func should be the recovered functions on previous step
# params should include the parameters to pass to each function
expectations_config = [
    {"func": expect_unique, "params": {"column": "pk_column"}},
    {"func": expect_not_null, "params": {"column": "pk_column"}},
    # {"func": expect_length_equal, "params": {"column": "pk_column", "value": 555}},
]

In [0]:
# Register Teams action
# How to create a Microsoft Teams Webhook: https://learn.microsoft.com/en-us/microsoftteams/platform/webhooks-and-connectors/how-to/add-incoming-webhook?tabs=newteams%2Cdotnet

from great_expectations.checkpoint import MicrosoftTeamsNotificationAction
gx_runner.register_action(
    name="teams_notify",
    action_class=MicrosoftTeamsNotificationAction,
    default_params={
        "name": "send_teams_notification_on_failed_expectations",
        "teams_webhook": dbutils.secrets.get(scope="scope_name", key="teams_webhook_url"), # to be replaced with your webhook
        "notify_on": "failure"
    }
)

# Store actions as list
action_list = [gx_runner.build_action("teams_notify")]

The majority of GX objects need a name/string to be identified. These names are arbitrary, for example:
* data_src_name = "data_src_test"
* data_asset_name = "data_asset_test" (_can be same/add other data asset_)
* batch_definition_name = "test_batch" (_can be other batch, e.g. for checking other expectations_)
* suite_name = "test_suite" (_can be same/add other Suite_)
* validation_definition_name = "validation_test" (_can add other validation definition_)
* checkpoint_name = "test_checkpoint" (_can be other checkpoint_)

In [0]:
# Run tests
df_results = gx_runner.run_tests(df,
                                 expectations_config,
                                 action_list=action_list, 
                                 data_src_name= "data_src_test", 
                                 data_asset_name= "data_asset_test", 
                                 batch_definition_name= "test_batch", 
                                 suite_name= "test_suite",
                                 validation_definition_name= "validation_test",
                                 checkpoint_name= "test_checkpoint"
                                 )

In [0]:
display(df_results)

### Interesting use cases:

* Using SQL to define a [Custom Expectation](https://docs.greatexpectations.io/docs/core/customize_expectations/use_sql_to_define_a_custom_expectation/) 
* Define a [Multi-source Expectation](https://docs.greatexpectations.io/docs/core/customize_expectations/define_a_multi_source_expectation/)

Docs [reference](https://docs.greatexpectations.io/docs/core/introduction/gx_overview/)